In [ ]:
from arcgis.gis import GIS
from tqdm.notebook import tqdm
import time
import re

def login_via_pro():
    """Log in using ArcGIS Pro session (SSO)"""
    return GIS("pro")

def get_all_items(user, max_items=1000):
    """Retrieve all items from user's root and folders"""
    all_items = user.items(max_items=max_items)
    for folder in user.folders:
        folder_items = user.items(folder=folder['title'], max_items=max_items)
        all_items.extend(folder_items)
    return all_items

def clean_title(title):
    """Clean item title: lowercase, replace spaces, collapse underscores"""
    title = title.lower().replace(" ", "_")
    return re.sub(r'_+', '_', title)

def rename_items(items):
    """Rename items with cleaned titles, skipping unsupported types"""
    renamed, skipped, failed = 0, 0, 0

    print("\n🔧 Starting renaming process (skipping Web Map & Web Experience):")
    for item in tqdm(items, desc="Renaming items"):
        original_title = item.title
        new_title = clean_title(original_title)

        if item.type in ["Web Map", "Web Experience"]:
            print(f"⏭️ Skipping '{original_title}' ({item.type}) — renaming not supported")
            skipped += 1
            continue

        if new_title != original_title:
            for attempt in range(3):
                try:
                    item.update(item_properties={"title": new_title})
                    print(f"✅ Renamed '{original_title}' → '{new_title}'")
                    renamed += 1
                    break
                except Exception as e:
                    print(f"⚠️ Attempt {attempt+1} failed for '{original_title}': {e}")
                    time.sleep(1)
            else:
                print(f"❌ Failed to rename '{original_title}' after 3 attempts")
                failed += 1

    print("\n📊 Rename Summary:")
    print(f"✅ Successfully renamed: {renamed}")
    print(f"⏭️ Skipped (Web Map & Web Experience): {skipped}")
    print(f"❌ Failed after retries: {failed}")

def main():
    gis = login_via_pro()
    user = gis.users.me
    items = get_all_items(user)
    print(f"\n📦 Total items retrieved: {len(items)}")
    rename_items(items)

if __name__ == "__main__":
    main()